In [1]:
!pip install zhon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 99 kB 2.5 MB/s 
  Created wheel for zhon: filename=zhon-1.1.5-py3-none-any.whl size=84322 sha256=64f0e600bca1a37bf08c02572d88454643a950d367f60ee491242d29ac1a6e0d
  Stored in directory: /root/.cache/pip/wheels/d0/56/17/2675c4c7413a72bf173062e8d0a16503e3b2607745aa84988d
Successfully built zhon


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 5.1 MB/s 


In [5]:
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer

In [ ]:
######test##############
import numpy as np
import pandas as pd
import time
import re

In [ ]:
import transformers
# import shap
import jieba
import re
from zhon.hanzi import punctuation
import torch

In [ ]:
df_ = pd.read_csv("/content/drive/MyDrive/guidao/data_3_4/df_ls.csv")
df_

,content,labels
0,我歌舞团明年去日访问据新华社东京25日电 中国人民对外文化协会代表孙平化25日下午在东京同日...,0
1,各国舆论欢呼开罗大会成就本报综合报道 据新华社讯：亚非人民团结大会胜利闭幕和它通过的一系列的...,0
2,——访兰州友谊饭店特级招待员胡春鉴本报记者 万贵生 刘宏10月中旬的一天，我们慕名访问了兰州...,0
3,新华社北京9月12日电（钱春弦、沈秋欢）日本外务报告官高岛肇久12日在此间召开的新闻吹风...,0
4,——基廷新政府百日外交本报记者史宗星从３月１３日大选揭晓那天算起，澳大利亚基廷新内阁执政已有...,0
...,...,...
22495,本报记者苗风用“北大荒”这个名称，来形容今天的黑龙江东北部地区已经有些不恰当了。在北至饶河县...,0
22496,广岛原子弹受害者静坐示威新华社7日讯 广岛消息：美国在广岛投掷的第一枚原子弹的五十名受害者昨...,0
22497,本报驻日本记者于青 日本民主党党首小泽一郎在11日下午正式宣布辞去党首职务。日本最大在...,0
22498,要求日政府放宽对华贸易限制新华社东京5月30日电日中经济贸易中心机关报《业务联络报》5月19...,0


In [ ]:
df_sample = df_.iloc[16200:16400,:]
df_sample

,content,labels
16200,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,0
16201,本报讯 社会各界向中国儿童少年活动中心首次赠礼仪式5月22日上午在北京官园举行。人大常委会副...,0
16202,李念慈随着经济建设必须依靠科学技术、科学技术必须面向经济建设方针的深入贯彻，人们越来越认识到...,0
16203,【新华社北京六日电】据塔斯社四日电引东京消息：旅居东京的朝鲜侨民和日本工人、学生约一万人，在...,0
16204,马晓春首盘胜柳时熏本报北京５月２０日讯记者李长云报道：中国棋院消息：马晓春九段今天在日本举行...,0
...,...,...
16395,新华社15日讯 中国亚非团结委员会主席廖承志今天上午接见了正在我国访问的日本哲学家山崎谦、寺...,0
16396,本报东京11月10日电 （记者于青）日本政府10日上午召开阿富汗和巴基斯坦援助措施阁僚会...,0
16397,本报记者刘硕阳 经过短暂的休整之后，俄罗斯世界杯于北京时间6月30日晚重燃战火，开始了...,0
16398,本报长春1月5日电 （记者刘亮明、孔祥武）国家加大投资拉动内需的决策，再次为吉林经济结构...,0


In [ ]:
df_sample.index = range(len(df_sample))
df_sample

,content,labels
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,0
1,本报讯 社会各界向中国儿童少年活动中心首次赠礼仪式5月22日上午在北京官园举行。人大常委会副...,0
2,李念慈随着经济建设必须依靠科学技术、科学技术必须面向经济建设方针的深入贯彻，人们越来越认识到...,0
3,【新华社北京六日电】据塔斯社四日电引东京消息：旅居东京的朝鲜侨民和日本工人、学生约一万人，在...,0
4,马晓春首盘胜柳时熏本报北京５月２０日讯记者李长云报道：中国棋院消息：马晓春九段今天在日本举行...,0
...,...,...
195,新华社15日讯 中国亚非团结委员会主席廖承志今天上午接见了正在我国访问的日本哲学家山崎谦、寺...,0
196,本报东京11月10日电 （记者于青）日本政府10日上午召开阿富汗和巴基斯坦援助措施阁僚会...,0
197,本报记者刘硕阳 经过短暂的休整之后，俄罗斯世界杯于北京时间6月30日晚重燃战火，开始了...,0
198,本报长春1月5日电 （记者刘亮明、孔祥武）国家加大投资拉动内需的决策，再次为吉林经济结构...,0


In [ ]:
def cut_word(text):
    text = re.sub('[^\u4e00-\u9fa5]+', '', text)
    # 精准模式
    jieba_list = jieba.cut(text, cut_all=False)#分词
    return jieba_list
df_sample['text'] = df_sample.content.apply(lambda x: " ".join(cut_word(x)))
df_sample['nopunc'] = df_sample.text.apply(lambda x:re.sub("[{}]+".format(punctuation), "", x))#利用正则表达式去掉标点符号
df_sample

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.947 seconds.
Prefix dict has been built successfully.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,content,labels,text,nopunc
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,0,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...
1,本报讯 社会各界向中国儿童少年活动中心首次赠礼仪式5月22日上午在北京官园举行。人大常委会副...,0,本报讯 社会各界 向 中国 儿童 少年 活动中心 首次 赠礼 仪式 月 日 上午 在 北京 ...,本报讯 社会各界 向 中国 儿童 少年 活动中心 首次 赠礼 仪式 月 日 上午 在 北京 ...
2,李念慈随着经济建设必须依靠科学技术、科学技术必须面向经济建设方针的深入贯彻，人们越来越认识到...,0,李念慈 随着 经济 建设 必须 依靠 科学技术 科学技术 必须 面向 经济 建设 方针 的 ...,李念慈 随着 经济 建设 必须 依靠 科学技术 科学技术 必须 面向 经济 建设 方针 的 ...
3,【新华社北京六日电】据塔斯社四日电引东京消息：旅居东京的朝鲜侨民和日本工人、学生约一万人，在...,0,新华社 北京 六日电 据 塔斯社 四日电 引 东京 消息 旅居 东京 的 朝鲜 侨民 和 日...,新华社 北京 六日电 据 塔斯社 四日电 引 东京 消息 旅居 东京 的 朝鲜 侨民 和 日...
4,马晓春首盘胜柳时熏本报北京５月２０日讯记者李长云报道：中国棋院消息：马晓春九段今天在日本举行...,0,马晓春 首盘 胜 柳时熏 本报 北京 月 日 讯 记者 李长云 报道 中国棋院 消息 马晓春...,马晓春 首盘 胜 柳时熏 本报 北京 月 日 讯 记者 李长云 报道 中国棋院 消息 马晓春...
...,...,...,...,...
195,新华社15日讯 中国亚非团结委员会主席廖承志今天上午接见了正在我国访问的日本哲学家山崎谦、寺...,0,新华社 日讯 中国 亚非 团结 委员会 主席 廖承志 今天上午 接见 了 正在 我国 访问 ...,新华社 日讯 中国 亚非 团结 委员会 主席 廖承志 今天上午 接见 了 正在 我国 访问 ...
196,本报东京11月10日电 （记者于青）日本政府10日上午召开阿富汗和巴基斯坦援助措施阁僚会...,0,本报 东京 月 日电 记者 于青 日本政府 日 上午 召开 阿富汗 和 巴基斯坦 援助 措施...,本报 东京 月 日电 记者 于青 日本政府 日 上午 召开 阿富汗 和 巴基斯坦 援助 措施...
197,本报记者刘硕阳 经过短暂的休整之后，俄罗斯世界杯于北京时间6月30日晚重燃战火，开始了...,0,本报记者 刘硕阳 经过 短暂 的 休整 之后 俄罗斯 世界杯 于 北京 时间 月 日晚 重燃...,本报记者 刘硕阳 经过 短暂 的 休整 之后 俄罗斯 世界杯 于 北京 时间 月 日晚 重燃...
198,本报长春1月5日电 （记者刘亮明、孔祥武）国家加大投资拉动内需的决策，再次为吉林经济结构...,0,本报 长春 月 日电 记者 刘亮明 孔祥武 国家 加大 投资 拉动 内需 的 决策 再次 为...,本报 长春 月 日电 记者 刘亮明 孔祥武 国家 加大 投资 拉动 内需 的 决策 再次 为...


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device

device(type='cpu')

In [ ]:
# 加载 BertForSequenceClassification, 预训练 BERT 模型 + 顶层的线性分类层 
model = BertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/guidao/result_6", # 小写的 12 层预训练模型
    num_labels = 2, # 分类数 --2 表示二分类
                    # 你可以改变这个数字，用于多分类任务  
#     output_attentions = False, # 模型是否返回 attentions weights.
#     output_hidden_states = False, # 模型是否返回所有隐层状态.
)
model.to(device)
model.eval()
model.zero_grad()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/guidao/result_6')

In [ ]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.logits

def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
#     pred = pred[position]
    return pred.max(1).values

In [ ]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [ ]:
def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(text_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = model.bert.embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = model.bert.embeddings(ref_input_ids, token_type_ids=ref_token_type_ids, position_ids=ref_position_ids)
    
    return input_embeddings, ref_input_embeddings

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [ ]:
def split_chunk(text): #为什么要split_chunk, 512的数值有意义吗
    str = text
#     str = re.sub(r"\s+", "", str)
    str = str.strip()
    n = 512
    chunks = []
    i = 0
    while i < len(str):
        if i+n < len(str):
            chunks.append(str[i:i+n])
        else:
            chunks.append(str[i:len(str)])
        i += n
    return(chunks)

lig = LayerIntegratedGradients(squad_pos_forward_func, model.bert.embeddings)
final_result1 = pd.DataFrame()
start = time.perf_counter()
for j in range(len(df_sample.nopunc)): 
    #for overall piece of news
    chunk_sample=split_chunk(df_sample.nopunc[j])
    lst_shapvalue=[]
    result_lst=[]
    print('now is ',j, '-th news')
    f1 = pd.DataFrame()
    for i in range(len(chunk_sample)):
        print([chunk_sample[i]])
        print(len(chunk_sample[i]))
        text = chunk_sample[i]
        #############
        input_ids, ref_input_ids, sep_id = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
        token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
        position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
        attention_mask = construct_attention_mask(input_ids)

        indices = input_ids[0].detach().tolist()
        all_tokens = tokenizer.convert_ids_to_tokens(indices)
        
        attributions, delta = lig.attribute(inputs=input_ids,
                                  baselines=ref_input_ids,
                                  additional_forward_args=(token_type_ids, position_ids, attention_mask, 0),
                                  return_convergence_delta=True)
        
        attributions_sum = summarize_attributions(attributions)
        
        lst_shapvalue.append(attributions_sum.numpy())
        
#         print(attributions_sum)
        
        result_lst.append(pd.DataFrame(all_tokens,columns=['Char'])) #first 0=i
        result_lst[i]['Score'] = attributions_sum.numpy()
#         result_lst[i] = result_lst[i].iloc[1:-1, :]
        if len(chunk_sample) > 1:
            if i == 0:
                result_lst[i] = result_lst[i].iloc[:-1, :]
            elif i == (len(chunk_sample)-1):
                result_lst[i] = result_lst[i].iloc[1:, :]
            else:
                result_lst[i] = result_lst[i].iloc[1:-1, :]
        final_result=pd.concat(result_lst)
        
    final_result1=pd.concat([final_result1, final_result])
    

end = time.perf_counter()
print('Running time: %s Seconds' % (end - start))

final_result1

now is  0 -th news
['各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争 不是 孤立 的 哈尔滨 据 新华社 哈尔滨 日电 哈尔滨 青年 和 各界 人民 三万多 人 今天下午 举行 了 集会 和 示威游行 坚决 支持 土耳其 人民 的 爱国 正义斗争 中共 黑龙江 省委 书记处 书记 强晓初 省委常委 宣传部长 于林 省委常委 统战部长 张瑞麟 中共 哈尔滨市委 书记处 书记 林肖 硖 等 黑龙江省 和 哈尔滨市 的 党政 领导人 各 民主党派 地方 组织 和 各 人民团体 的 负责人 都 参加 了 大会 苏联 驻 哈尔滨市 领事馆 的 副 领事 库 尔巴 茨 基 和 正在 哈尔滨市 访问 的 越南 电影 工作者 代表团 也 参加 了 大会 共青团 黑龙江 省委书记 吴亮 璞 在 大会 上 讲 了 话 他 说 土耳其 人民 的 爱国 正义斗争 绝不 是 孤立 的 中国 人民 和 全世界 爱好和平 的 人民 对 他们 表示 了 坚决 的 支持 只要 他们 坚持 斗争 他们 一定 能够 取得 最后 胜利 吴亮 璞 接着 号召 全省 青年 发扬 五四 爱国 反帝 斗争 的 光荣传统 坚决 支持 ']
512
['亚洲 非洲 拉丁美洲 的 民族 民主运动 各 民主党派 地方 组织 和 各 人民团体 的 负责人 以及 各界 代表 都 在 会上 讲 了 话 会后 举行 了 示威游行 支持 被压迫 民族 的 反帝 爱国 斗争 长春 据 新华社 长春 日电 吉林省 和 长春市 各界 青年 和 人民 一万五千 人 今日 下午 举行 集会 支持 土耳其 人民 的 爱国 正义斗争 参加 大会 的 有 吉林省 副省长 徐寿轩 王奂如 中共 吉林省 委 宣传部长 宋振庭 中共 长春市委 书记处 书记 李都 以及 吉林省 与 长春市 各 群众团体 民主党派 负责人 应 共青团 吉林省 委 邀请 前来 参加 吉林省 五一国际劳动节 庆祝 活动 的 朝鲜 咸镜北道 两江道 慈江道 青年 代表团 的 全体 代表 也 参加 了 大会 共青团 吉林 省委书记 蒋谷峰 在 会上 讲话 他 说 今天 是 五四运动 四十 一周年 纪念日 有着 反帝 斗争 和 五四 光荣传统 的 中国 人民 和 青年 对于 土耳其 人民 的 爱国 正义斗争 表示 极

,Char,Score
0,[CLS],0.000000
1,各,0.041208
2,地,0.040715
3,群,0.084992
4,众,0.018618
...,...,...
76,目,0.016642
77,不,-0.015874
78,忍,-0.018150
79,睹,-0.026204


In [ ]:
word_t = pd.DataFrame(columns=['text'])

In [ ]:
final_result1 = final_result1[final_result1['Char'] != '[UNK]']
final_result1

,Char,Score
0,[CLS],0.000000
1,各,0.041208
2,地,0.040715
3,群,0.084992
4,众,0.018618
...,...,...
76,目,0.016642
77,不,-0.015874
78,忍,-0.018150
79,睹,-0.026204


In [ ]:
final_result1.index=range(len(final_result1))
final_result1

,Char,Score
0,[CLS],0.000000
1,各,0.041208
2,地,0.040715
3,群,0.084992
4,众,0.018618
...,...,...
230182,目,0.016642
230183,不,-0.015874
230184,忍,-0.018150
230185,睹,-0.026204


In [ ]:
len_f = len(final_result1)
ij = 0
jn = 0
while(ij < len_f):
    if final_result1.iloc[ij, 0] == '[CLS]':
        txt = ''
        ij += 1
        while(ij < len_f and final_result1.iloc[ij, 0] != '[SEP]'):
            txt += final_result1.iloc[ij, 0]
            ij += 1
        word_t.loc[jn, :] = [txt]
        jn += 1
    else:
        ij += 1
word_t 

,text
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...
1,本报讯社会各界向中国儿童少年活动中心首次赠礼仪式月日上午在北京官园举行人大常委会副委员长朱学...
2,李念慈随着经济建设必须依靠科学技术科学技术必须面向经济建设方针的深入贯彻人们越来越认识到要振...
3,新华社北京六日电据塔斯社四日电引东京消息旅居东京的朝鲜侨民和日本工人学生约一万人在反对重新武...
4,马晓春首盘胜柳时熏本报北京月日讯记者李长云报道中国棋院消息马晓春九段今天在日本举行的中日围棋...
...,...
195,新华社日讯中国亚非团结委员会主席廖承志今天上午接见了正在我国访问的日本哲学家山崎谦寺泽恒信高...
196,本报东京月日电记者于青日本政府日上午召开阿富汗和巴基斯坦援助措施阁僚会议确定了自年度起年内提...
197,本报记者刘硕阳经过短暂的休整之后俄罗斯世界杯于北京时间月日晚重燃战火开始了更加紧张激烈的淘汰...
198,本报长春月日电记者刘亮明孔祥武国家加大投资拉动内需的决策再次为吉林经济结构调整注入动力近年来...


In [ ]:
final_df = final_result1[final_result1['Char'] != '[CLS]']
final_df = final_df[final_df['Char'] != '[SEP]']
final_df

,Char,Score
1,各,0.041208
2,地,0.040715
3,群,0.084992
4,众,0.018618
5,集,0.054101
...,...,...
230181,人,-0.021254
230182,目,0.016642
230183,不,-0.015874
230184,忍,-0.018150


In [ ]:
final_result1 = final_df
final_result1

,Char,Score
1,各,0.041208
2,地,0.040715
3,群,0.084992
4,众,0.018618
5,集,0.054101
...,...,...
230181,人,-0.021254
230182,目,0.016642
230183,不,-0.015874
230184,忍,-0.018150


In [ ]:
word_t['text_new'] = word_t.text.apply(lambda x: " ".join(cut_word(x)))
word_t

,text,text_new
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...
1,本报讯社会各界向中国儿童少年活动中心首次赠礼仪式月日上午在北京官园举行人大常委会副委员长朱学...,本报讯 社会各界 向 中国 儿童 少年 活动中心 首次 赠礼 仪式 月 日 上午 在 北京 ...
2,李念慈随着经济建设必须依靠科学技术科学技术必须面向经济建设方针的深入贯彻人们越来越认识到要振...,李念慈 随着 经济 建设 必须 依靠 科学技术 科学技术 必须 面向 经济 建设 方针 的 ...
3,新华社北京六日电据塔斯社四日电引东京消息旅居东京的朝鲜侨民和日本工人学生约一万人在反对重新武...,新华社 北京 六日电 据 塔斯社 四日电 引 东京 消息 旅居 东京 的 朝鲜 侨民 和 日...
4,马晓春首盘胜柳时熏本报北京月日讯记者李长云报道中国棋院消息马晓春九段今天在日本举行的中日围棋...,马晓春 首盘 胜 柳时熏 本报 北京 月 日 讯 记者 李长云 报道 中国棋院 消息 马晓春...
...,...,...
195,新华社日讯中国亚非团结委员会主席廖承志今天上午接见了正在我国访问的日本哲学家山崎谦寺泽恒信高...,新华社 日讯 中国 亚非 团结 委员会 主席 廖承志 今天上午 接见 了 正在 我国 访问 ...
196,本报东京月日电记者于青日本政府日上午召开阿富汗和巴基斯坦援助措施阁僚会议确定了自年度起年内提...,本报 东京 月 日电 记者 于青 日本政府 日 上午 召开 阿富汗 和 巴基斯坦 援助 措施...
197,本报记者刘硕阳经过短暂的休整之后俄罗斯世界杯于北京时间月日晚重燃战火开始了更加紧张激烈的淘汰...,本报记者 刘硕阳 经过 短暂 的 休整 之后 俄罗斯 世界杯 于 北京 时间 月 日晚 重燃...
198,本报长春月日电记者刘亮明孔祥武国家加大投资拉动内需的决策再次为吉林经济结构调整注入动力近年来...,本报 长春 月 日电 记者 刘亮明 孔祥武 国家 加大 投资 拉动 内需 的 决策 再次 为...


In [ ]:
word_t['Phrase']=word_t.text_new.apply(lambda x: x.split(' '))
df_t = word_t.explode("Phrase")

df_t

,text,text_new,Phrase
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,集会
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,表示
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,支持
...,...,...,...
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,其状
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,之
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,惨
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,令人


In [ ]:
def split(word):
    return [char for char in word]

df_t['Char'] = df_t.Phrase.apply(lambda x:split(x))
df_t=df_t.explode('Char') #以“Char”列进行爆炸，分开每一个词
# df3all.reset_index(level=0)
df_t

,text,text_new,Phrase,Char
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,各
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,地
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,群
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,众
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,集会,集
...,...,...,...,...
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,令人,人
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,目
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,不
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,忍


In [ ]:
df_t['Score'] = list(final_result1.iloc[:, 1])
df_t

,text,text_new,Phrase,Char,Score
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,各,0.041208
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,地,0.040715
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,群,0.084992
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,众,0.018618
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,集会,集,0.054101
...,...,...,...,...,...
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,令人,人,-0.021254
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,目,0.016642
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,不,-0.015874
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,忍,-0.018150


In [ ]:
#设置词的长度
df_t['len_phase'] = 1
list_len_ph = []
for i in range(len(df_t)):
#     print(df3.iloc[i]['Phrase'], len(df3.iloc[i]['Phrase']))
    list_len_ph.append(len(df_t.iloc[i]['Phrase']))
df_t['len_phase'] = list_len_ph
df_t

,text,text_new,Phrase,Char,Score,len_phase
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,各,0.041208,2
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,地,0.040715,2
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,群,0.084992,2
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,众,0.018618,2
0,各地群众集会表示支持土耳其人民争取斗争的胜利土耳其人民的正义斗争不是孤立的哈尔滨据新华社哈尔...,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,集会,集,0.054101,2
...,...,...,...,...,...,...
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,令人,人,-0.021254,2
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,目,0.016642,4
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,不,-0.015874,4
199,日军在河南信阳的大屠杀蔡静远日本帝国主义发动的侵华战争给中国人民造成了深重的灾难仅就屠杀中国...,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,目不忍睹,忍,-0.018150,4


In [ ]:
#按每个词的长度进行分数的加和
df3_test = pd.DataFrame(columns=['nopunc',  'Phrase', 'Score'])
n = 1
tmi = 0

while tmi < len(df_t):
    n = df_t.iloc[tmi]['len_phase']   
    s_l = s_r = 0
    for j in range(n):
        s_l = s_l + df_t.iloc[tmi+j]['Score']
    df3_test.loc[tmi] = [df_t.iloc[tmi]['text_new'], df_t.iloc[tmi]['Phrase'], s_l]
    tmi = tmi + n

df3_test.index = range(len(df3_test))
df3_test

,nopunc,Phrase,Score
0,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,各地,0.081923
1,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,群众,0.103609
2,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,集会,0.088336
3,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,表示,0.066173
4,各地 群众 集会 表示 支持 土耳其 人民 争取 斗争 的 胜利 土耳其 人民 的 正义斗争...,支持,0.017861
...,...,...,...
119624,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,其状,0.029284
119625,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,之,-0.002469
119626,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,惨,0.025689
119627,日军 在 河南 信阳 的 大屠杀 蔡静远 日本 帝国主义 发动 的 侵华 战争 给 中国 人...,令人,-0.033600


In [ ]:
dff = df3_test.groupby('nopunc')['Score'].apply(list).reset_index(name='score')
dff

,nopunc,score
0,一九九 年 敦煌学 国际 学术 讨论会 结束 据 新华社 兰州 月 日电 记者 曹 永安 长...,"[-0.004782752998027833, 0.024302387994227907, ..."
1,一九二八年 在 以 蒋介石 为首 的 新 军阀 北伐 进军 济南 之际 日本政府 为 保护 ...,"[-0.06363706402358398, -0.003251662901702968, ..."
2,一些 学者 强烈 反对 首相 参拜 靖国神社 本报 东京 月 日电 记者 管克江 报道 日本...,"[-0.2280307059353051, 0.17455454792248248, 0.5..."
3,一论 学习 贯彻 党 的 十五届 五中全会 精神 本报 评论员 党 的 十五届 五中全会 通...,"[0.2776298779597024, 0.1618719505116435, 0.247..."
4,上海 渐成 国际 文化 交流中心 新华社 记者 赵兰英 本报记者 郭伟成 解放日报 记者 端...,"[-0.11484086699275667, 0.9906504514294187, 0.1..."
...,...,...
195,长野 成功 传 圣火 综合 本报 和 新华社 记者 报道 当地 时间 月 日 时分 北京 奥...,"[-0.006820085957020636, 0.20549567223096718, 0..."
196,阮仲永 大使 的 讲话 新华社 一九七五年 五月 二日 讯 阮仲永 大使 在 首都 人民 庆...,"[0.11493441379037843, 0.0933145692717925, 0.00..."
197,非洲 国家 代表 批评 美英 保护 南非 当局 利益 本报 联合国 月 日电 记者 刘开宸 ...,"[-0.020459823947973053, 0.030814662216571805, ..."
198,首都 各界 集会 庆祝 全非 人民 大会 和 亚非 团结 理事会 成就 并 欢迎 非洲 朋友...,"[-0.018776767906558925, -0.04512491864238127, ..."


In [ ]:
dff.to_csv("/content/drive/MyDrive/guidao/data_3_4/score31_ls_df.csv",index=False)